# W2D5 - PyTorch 数据加载：`Dataset` 与 `DataLoader`**

**今日目标:**

1.  理解 `Dataset` 类的作用，并掌握 `__len__` 和 `__getitem__` 这两个“魔法方法”。
2.  理解 `DataLoader` 的作用，并掌握 `batch_size`, `shuffle`, `num_workers` 三个核心参数的含义。
3.  学会如何使用 PyTorch 内置的 `torchvision` 来加载 MNIST 数据集。

## `Dataset`：“数据在哪里，如何取？”

`Dataset` 是一个 PyTorch 抽象类，用于\*\*“包装”\*\*你的数据集。它只关心两件事：

1.  **“我们一共有多少数据？”**
2.  **“如果我要第 `i` 条数据，你怎么给我？”**

为了实现这一点，你创建的任何 `Dataset` 类都**必须**实现两个“魔法方法” (dunder methods)：

  * **`__len__(self)`:**

      * **职责：** 返回数据集中样本的总数。
      * **被谁调用？** `DataLoader` 需要知道总长度，以便计算出总共有多少个批次。

  * **`__getitem__(self, idx)`:**

      * **职责：** **根据索引 `idx`（一个数字）返回一条数据**。这是 `Dataset` 的核心。
      * **被谁调用？** `DataLoader` 会自动地、一条一条地调用这个方法来获取数据，然后将它们打包成一个批次。
      * **返回值：** 通常是一个元组 (tuple)，例如 `(data, label)`。比如 `(一张图片的 Tensor, 这张图片的标签)`。

**代码实践 1：自定义一个 `Dataset` (概念)**

我们来创建一个虚拟的 `MyFakeDataset`，它有 1000 条数据，每条数据是 `(一个随机数, 它的平方)`。

In [1]:
import torch
from torch.utils.data import Dataset

# 1. 创建我们自己的 Dataset 类，它必须继承 Dataset
class MyFakeDataset(Dataset):
    
    # 2. 实现 __init__ (可选), 用来初始化数据源
    def __init__(self, num_samples):
        # 假设我们的数据是 0 到 num_samples-1
        self.total_samples = num_samples
        # (在真实项目中，这里可能是加载一个 .csv 文件或一个文件名列表)

    # 3. [必须实现] __len__: 返回数据集总长度
    def __len__(self):
        return self.total_samples

    # 4. [必须实现] __getitem__: 根据索引 idx 返回一条数据
    def __getitem__(self, idx):
        # 在真实项目中，这里可能是：
        # 1. 根据 idx 找到图片路径
        # 2. 读取图片
        # 3. 对图片进行变换 (transform)
        # 4. 返回 (图片, 标签)
        
        # 在我们的虚拟例子中：
        data = torch.tensor(float(idx))
        label = torch.tensor(float(idx**2))
        
        return data, label # 必须返回一个 (数据, 标签) 对

# --- 测试代码 ---
fake_data = MyFakeDataset(num_samples=10)

# 测试 __len__
print(f"数据集总长度: {len(fake_data)}") # 自动调用 fake_data.__len__()

# 测试 __getitem__
# 让我们手动获取第 3 条数据 (idx=3)
sample_data, sample_label = fake_data[3] # 自动调用 fake_data.__getitem__(3)
print(f"第 3 条数据: {sample_data}, 标签: {sample_label}") # 应该是 3.0 和 9.0

数据集总长度: 10
第 3 条数据: 3.0, 标签: 9.0


## **`DataLoader`：“如何送，送多少？”**

如果你自己有了一个 `Dataset`，你当然可以用 `for` 循环来遍历它。但这样做效率极低（没有分批、没有打乱、没有多进程）。

`DataLoader` 就是一个强大的“数据加载器”，它接收你的 `Dataset` 对象，并自动帮你处理所有繁琐但重要的数据加载工作。

**`DataLoader` 的三大核心参数 (面试高频):**

1.  **`batch_size` (整数):**

      * **含义：** “批大小”。`DataLoader` 每次迭代（每个 step）返回多少条数据。
      * **为什么？** 我们用 SGD 及其变体进行训练，每次更新参数都需要一个“小批次 (mini-batch)”的数据。`batch_size=64` 意味着它会从 `Dataset` 中一次取 64 条数据（调用 64 次 `__getitem__`），将它们打包成一个大 `Tensor`（形状变为 `[64, ...]`) 再返回给你。

2.  **`shuffle=True` (布尔值):**

      * **含义：** **在每个 Epoch 开始时，是否要打乱数据的顺序。**
      * **为什么？** **在训练时，这几乎是必须的！** 如果不打乱，模型可能会学到数据的（无意义的）顺序，或者在面对特定顺序的数据时表现不佳，导致训练效果变差或过拟合。
      * **注意：** 在**验证集**和**测试集**上，**不需要** `shuffle=True`，因为评估时顺序不重要。

3.  **`num_workers` (整数, 默认为 0):**

      * **含义：** 使用多少个**子进程**来提前加载数据。
      * **为什么？** `__getitem__` 中可能包含耗时操作（如从磁盘读图、解码、做复杂的数据增强）。如果 `num_workers=0` (默认)，所有加载都在主进程中进行，GPU 训练时可能会“饿肚子”（即 GPU 算完了上一个批次，在等 CPU 加载下一个批次）。
      * **`num_workers > 0` (例如 `num_workers=4`)**：PyTorch 会启动 4 个子进程，在后台**并行地、提前**加载好接下来的几个批次，存入内存。这样 GPU 一算完，就能立刻拿到新数据，极大提升训练效率。
      * **WSL/Linux 用户建议：** 设为 4 或 8。

## **W2D5 代码实践：使用内置的 MNIST 数据集**

幸运的是，对于像 MNIST 这样的标准数据集，`torchvision` 已经帮我们写好了 `Dataset`，我们只需要直接使用它，并用 `DataLoader` 包装起来。

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# --- 准备工作 ---

# 1. 定义数据预处理 (Transform)
# 这是一个非常重要的步骤
# PyTorch 的 Dataset 在 __getitem__ 中会调用 transform
transform = transforms.Compose([
    transforms.ToTensor(), # 将 PIL Image 或 NumPy 数组 转换为 PyTorch Tensor (范围 [0, 1])
    transforms.Normalize((0.5,), (0.5,)) # 将 [0, 1] 范围的 Tensor 标准化到 [-1, 1] 范围
])

# 2. [Dataset] 加载 torchvision 内置的 MNIST 训练集
# root='./data': 数据下载/存放在哪里
# train=True: 我们要训练集
# download=True: 如果 './data' 里没有，就自动下载
# transform=transform: 应用我们上面定义的预处理
train_dataset = datasets.MNIST(
    root='../../data', 
    train=True, 
    download=True, 
    transform=transform
)

# 3. [DataLoader] 使用 DataLoader 包装我们的训练集
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=64,   # 每次给我们 64 张图片
    shuffle=True,    # 每个 epoch 都打乱顺序
    num_workers=4    # 用 4 个子进程在后台加载数据 (在 Windows 本机上可能要设为 0)
                     # 在 WSL 中，4 通常是安全的
)

# --- 测试 DataLoader ---
print(f"训练集总样本数: {len(train_dataset)}")
print(f"DataLoader 将会产生多少个批次: {len(train_loader)}") # 60000 / 64 = 937.5 -> 938

# 4. 模拟一个训练循环，从 DataLoader 中取一个批次的数据
# next(iter(...)) 是一个标准方法，用来获取迭代器的第一个元素
data_batch, labels_batch = next(iter(train_loader))

# 5. 检查数据形状 (这是 Debug 的关键!)
print(f"\n--- 获取到的一个批次 ---")
print(f"数据批次 (data_batch) 的形状: {data_batch.shape}")
print(f"标签批次 (labels_batch) 的形状: {labels_batch.shape}")

# 预期输出：
# data_batch shape: [64, 1, 28, 28] 
# (64=batch_size, 1=通道数(灰度图), 28x28=图片尺寸)
# labels_batch shape: [64]
# (64个标签，对应64张图片)


Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw

训练集总样本数: 60000
DataLoader 将会产生多少个批次: 938



--- 获取到的一个批次 ---
数据批次 (data_batch) 的形状: torch.Size([64, 1, 28, 28])
标签批次 (labels_batch) 的形状: torch.Size([64])


## **W2D5 今日行动清单**

1.  **✅ 运行代码：** 亲手将上面两个代码实践块（特别是第二个）在你的环境中运行一遍。
2.  **✅ 检查下载：** 运行后，你会发现在你的代码同级目录下多了一个 `./data` 文件夹，里面就是下载的 MNIST 数据集。
3.  **✅ 理解形状：** **这是今天的核心！** 确保你完全理解为什么 `data_batch.shape` 是 `[64, 1, 28, 28]`，而 `labels_batch.shape` 是 `[64]`。
4.  **✅ 思考题 (面试模拟)：**
      * “在 `DataLoader` 中，`shuffle=True` 和 `num_workers` 参数分别有什么作用？为什么在训练时前者几乎是必须的？”
